In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [77]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import re
import nltk
from nltk.tokenize import word_tokenize

# تحميل الموارد اللازمة لـ NLTK
nltk.download('punkt')

# قائمة مخصصة من الكلمات الزائدة العربية
ARABIC_STOPWORDS = {
    'من', 'في', 'على', 'إلى', 'عن', 'مع', 'بين', 'حتى', 'منذ', 'عند', 'فوق', 'تحت',
    'أمام', 'خلف', 'هنا', 'هناك', 'كل', 'بعض', 'أي', 'هذا', 'هذه', 'تلك', 'ذلك',
    'الذي', 'التي', 'اللذين', 'اللاتي', 'إن', 'أن', 'لكن', 'و', 'أو', 'لا', 'لم',
    'لن', 'ما', 'ماذا', 'كيف', 'متى', 'أين', 'لماذا'
}

# تحميل نموذج تحليل المشاعر المحسن
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# دالة لتنظيف النصوص
def clean_text(text):
    # إزالة الرموز التعبيرية والرموز غير العربية
    text = re.sub(r'[^\u0621-\u064A\s0-9]', '', text)
    # تحويل إلى حروف صغيرة
    text = text.lower()
    # إزالة الكلمات الزائدة
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in ARABIC_STOPWORDS]
    return ' '.join(tokens)

# دالة لتحليل المشاعر مع عتبة ثقة
def analyze_sentiments(comments, confidence_threshold=0.7):
    results = []
    for comment in comments:
        cleaned_comment = clean_text(comment)
        result = sentiment_analyzer(cleaned_comment)[0]
        label = result['label']
        score = result['score']
        
        # تحويل التسميات إلى إيجابي/سلبي/محايد
        if label == 'positive' and score >= confidence_threshold:
            sentiment = 'إيجابي'
        elif label == 'negative' and score >= confidence_threshold:
            sentiment = 'سلبي'
        elif label == 'neutral' or score < confidence_threshold:
            sentiment = 'محايد'
        else:
            sentiment = 'محايد'  # في حالة عدم اليقين
        results.append({'تعليق': comment, 'تعليق_منظف': cleaned_comment, 'المشاعر': sentiment, 'الثقة': score})
    return pd.DataFrame(results)

# دالة لاستخراج الكلمات الشائعة
def get_common_words(df, sentiment, n=5):
    words = []
    for comment in df[df['المشاعر'] == sentiment]['تعليق_منظف']:
        words.extend(word_tokenize(comment))
    return Counter(words).most_common(n)

# دالة لإنشاء تقرير تلقائي محسن
def generate_report(df, output_dir="reports"):
    os.makedirs(output_dir, exist_ok=True)
    
    # إحصائيات
    sentiment_counts = Counter(df['المشاعر'])
    total_comments = len(df)
    positive_pct = (sentiment_counts['إيجابي'] / total_comments * 100) if 'إيجابي' in sentiment_counts else 0
    negative_pct = (sentiment_counts['سلبي'] / total_comments * 100) if 'سلبي' in sentiment_counts else 0
    neutral_pct = (sentiment_counts['محايد'] / total_comments * 100) if 'محايد' in sentiment_counts else 0
    
    # استخراج الكلمات الشائعة
    positive_words = get_common_words(df, 'إيجابي')
    negative_words = get_common_words(df, 'سلبي')
    neutral_words = get_common_words(df, 'محايد')
    
    # إنشاء التقرير
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    report_content = f"""
# تقرير تحليل المشاعر المحسن
## تاريخ التقرير: {timestamp}

### ملخص الإحصائيات
- **إجمالي التعليقات**: {total_comments}
- **إيجابية**: {sentiment_counts.get('إيجابي', 0)} ({positive_pct:.2f}%)
- **سلبية**: {sentiment_counts.get('سلبي', 0)} ({negative_pct:.2f}%)
- **محايدة**: {sentiment_counts.get('محايد', 0)} ({neutral_pct:.2f}%)

### التعليقات التفصيلية
| التعليق | المشاعر | الثقة |
|----------|----------|--------|
"""
    for _, row in df.iterrows():
        report_content += f"| {row['تعليق']} | {row['المشاعر']} | {row['الثقة']:.2f} |\n"

    # حفظ التقرير
    report_path = f"{output_dir}/sentiment_report_{timestamp}.md"
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report_content)
    
    # إنشاء رسم بياني
    plt.figure(figsize=(8, 6))
    sns.barplot(x=list(sentiment_counts.keys()), y=list(sentiment_counts.values()))
    plt.title("توزيع المشاعر في التعليقات")
    plt.xlabel("المشاعر")
    plt.ylabel("عدد التعليقات")
    plt_path = f"{output_dir}/sentiment_plot_{timestamp}.png"
    plt.savefig(plt_path)
    plt.close()
    
    return report_path, plt_path

# مثال للتعليقات
if __name__ == "__main__":
    comments = [
        "المنتج رائع وسهل الاستخدام! 😊",
        "خدمة العملاء سيئة جدًا وغير متعاونة 😡",
        "المنتج عادي، لا بأس به 😐",
        "تجربة ممتازة، سأشتري مرة أخرى 👍",
        "التوصيل تأخر كثيرًا 😞"
    ]
    
    # تحليل المشاعر
    df = analyze_sentiments(comments)
    
    # إنشاء التقرير
    report_path, plot_path = generate_report(df)
    print(f"تم إنشاء التقرير في: {report_path}")
    print(f"تم إنشاء الرسم البياني في: {plot_path}")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cuda:0


تم إنشاء التقرير في: reports/sentiment_report_2025-07-14_16-14-51.md
تم إنشاء الرسم البياني في: reports/sentiment_plot_2025-07-14_16-14-51.png


/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1765: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  order = pd.unique(vector)


In [76]:
import shutil

shutil.rmtree('/kaggle/working/reports')